In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5
import numpy as np
import time
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import analysis.switching as sw

In [3]:
from experiments.Pulse_Switching_CSHE_BER import BERExperiment

In [4]:
exp = BERExperiment()
exp.sample = "CSHE5 - C1R3"
exp.comment = "Bit Error Rate - AP to P - 9ns - Settle delay 200us - Reset 0.2V 5ns"
exp.polarity = -1 # -1: AP to P; 1: P to AP
exp.field.value = -0.0074
exp.pulse_duration.value = 9e-9 # Fixed
exp.reset_amplitude = 0.2
exp.reset_duration  = 5.0e-9
exp.settle_delay    = 200e-6
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C1R3\CSHE5-C1R3-AP2P_2016-07-21_BER_9ns.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
# attempts_list = [1 << int(x) for x in np.linspace(14,20,7)]
attempts_list = [1 << int(x) for x in [16,18,20,22,22,22,22]]
voltages_list = np.linspace(0.45,0.6,7)
print("Estimated time: {} min.".format(np.sum(attempts_list)/1e5))
t1 = [] # Keep track of time
t2 = []
for att, vol in tqdm_notebook(zip(attempts_list, voltages_list), total=len(voltages_list)):
    print("Now at ({},{}).".format(att,vol))
    t1.append(time.time())
    exp.pulse_voltage.value = vol
    exp.attempts.value = att
    exp.init_streams()
    exp.reset()
    exp.run_loop()
    t2.append(time.time())
    print("Elapsed time: {} min.".format((t2[-1]-t1[-1])/60))

exp.shutdown_instruments()
print("Total elapsed time: {} min.".format((t2[-1]-t1[0])/60))

Estimated time: 181.53472 min.
Now at (65536,0.45).
Elapsed time: 0.6508428970972697 min.
Now at (262144,0.475).
Elapsed time: 2.532727785905202 min.
Now at (1048576,0.5).
Elapsed time: 10.04906089703242 min.
Now at (4194304,0.525).
Elapsed time: 40.113547178109485 min.
Now at (4194304,0.55).
Elapsed time: 40.12255169550578 min.
Now at (4194304,0.575).
Elapsed time: 40.111667740345 min.
Now at (4194304,0.6).
Elapsed time: 40.117519724369046 min.

Total elapsed time: 173.6979179183642 min.


In [7]:
# Plot data
volts, data_mean = sw.load_BER_data(wr.filename)
fig = sw.plot_BER(volts, data_mean)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)
